## 5.horseIDとjockeyIDの取得

In [54]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import functions
import numpy as np
import importlib

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 30

In [ ]:

df = pd.read_pickle('2.2.add.日付_df.pickle')
race_id_list = df.index.unique()
print(type(race_id_list[0]))
# race_id_list = ["202101010101","202101010102"]
race_id_list
# df

In [ ]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      print(f"レースID: {_race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [ ]:
results_new = scrape_race_results(race_id_list)
results_new

----

## 6.取得したデータを成形して元のデータと結合

In [ ]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
print(results_new)
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)
# results

In [ ]:
print(type(results))
print(len(results))
print(type(df))
print(len(df))

In [ ]:
# pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [ ]:
# df_add_id.to_pickle("df_add_id.pickle")

----

## 7.idのリストを作成

In [ ]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
# horse_id_list = horse_id_list[:50]
# horse_id_list = ['2018105875',"2018102192"]

In [ ]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [ ]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付',encoding='EUC-JP')[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [ ]:
horse_results = scrape_horse_results(horse_id_list)

In [ ]:
horse_results

----

## 取得したデータを成形

In [ ]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [ ]:
# pd.options.display.max_rows = None

results

In [ ]:
# results.to_csv("3.2.horse_vs_data.csv")

In [ ]:
# time.sleep(2)
# df = pd.read_csv("3.2.horse_vs_data.csv")

In [ ]:
# print(df.columns)
# df = df.iloc[:,:29]
# print(df.columns)

----

## データの自動成形

In [55]:
importlib.reload(functions)

arr = [
       13238,13239,13334,13335,13958,13978,13993,14019,
       14020,14316,14317,14341,14342,14388,14548,14549,
       14555,14556,14718,14719,14555,14556,15056,15057,
       15129,14555,14556,15057,15129,15130,15316,15317,
       15378,15379,15425,15426,15466,15517,15518,15519,
       15557,15558,15638,15639,15665,15666,15799,15800,
       15838,15839,15859,15860
       ]

clean_index = list(map(lambda x: x-1,arr))
# l,l_length,columns_len = Molding(clean_index,"3.2.horse_vs_data.csv")

l,l_length,columns_len,おかしいリスト = functions.Molding_new(clean_index,"3.4.horse_vs_data.csv")
おかしいリスト

[]

In [ ]:
print(f"全体のlen{l_length}")
print(f"カラムのlen{columns_len}")

In [ ]:
for i in l :
  if len(i) == 32:pass
  else: print(len(i), i)

In [ ]:

add = np.array(l[1:]).reshape(l_length-1,32)


10秒

In [ ]:
df_add = pd.DataFrame(add, columns=l[0])
# print(df_add)

In [ ]:
df_add.to_csv('3.4.horse_vs_data.csv',index=False)

In [ ]:
# print(df["日付"].value_counts())
# print(df["開催"].value_counts())
# print(df["天気"].value_counts())
# print(df["R"].value_counts())
# print(df["レース名"].value_counts())
# print(df["映像"].value_counts())
# print(df["頭数"].value_counts())
# print(df["枠番"].value_counts())
# print(df["馬番"].value_counts())
# print(df["オッズ"].value_counts())
# print(df["人気"].value_counts())
# print(df["着順"].value_counts())
# print(df["騎手"].value_counts())
# print(df["斤量"].value_counts())